In [1]:
# ===== Cell 1: Imports & Config =====
import os, json, sqlite3
from pathlib import Path
from typing import Iterable, List, Dict, Tuple, Optional, Union

import numpy as np
import pandas as pd

# 可视化（可选）
import matplotlib.pyplot as plt

# ML（可选）
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, mean_absolute_error, r2_score

# ----------------- 配置 -----------------
DB = "data/landscape-analysis.db"      # 你的 SQLite
OUT_ROOT = Path("data/MethodComparisonRuns")
PROBLEM_IDS: Iterable[int] = list(range(0, 8))  # 需要分析的 problem 范围
TARGET_EPOCHS = 100                    # 你希望的“跑满” epoch 数判定阈值

# 可选：如果你已经确定 feature 表名和键列，可以在这里填
FEATURE_TABLE_NAME = None              # 例如: "mla_features"
FEATURE_KEY_COLUMN = None              # 例如: "problem_id" 或 "nn_problem_id"


In [2]:
# ===== Cell 2: DB helpers & gen_log readers =====

def _rows_for_problems(db: str, pids: Iterable[int]) -> List[Tuple]:
    """从 DB 取 run 清单 (run_id, problem_type, problem_id, algo_name, result_filename)"""
    pids = list(pids)
    if not pids:
        return []
    placeholders = ",".join("?" for _ in pids)
    q = f"""
    SELECT r.id, r.problem_type, r.problem_id, a.name, r.result_filename
    FROM runs r
    JOIN algorithm a ON a.id = r.algorithm_id
    WHERE r.problem_type='nn' AND r.problem_id IN ({placeholders})
    ORDER BY r.problem_id, a.name, r.id;
    """
    with sqlite3.connect(db) as con:
        return con.execute(q, pids).fetchall()

def _read_gen_log(log_path: Path) -> pd.DataFrame:
    """
    读取 gen_log.csv，兼容你“新列集”：
      gen,n_evals,epoch,batch,best_f,best_acc,avg_acc,wall_time,epoch_acc_full,x_file,x_epoch_file
    也兼容旧格式（缺的列自动补 NA）。
    """
    # 允许列 superset，但下游分析只用到以下这些：
    cols = ["gen","n_evals","epoch","batch","best_f","best_acc","avg_acc",
            "wall_time","epoch_acc_full","x_file","x_epoch_file"]
    if not log_path.exists():
        return pd.DataFrame(columns=cols)

    df = pd.read_csv(log_path)
    for c in cols:
        if c not in df.columns:
            df[c] = pd.NA

    for c in ["gen","n_evals","epoch","batch","best_f","best_acc","avg_acc","wall_time","epoch_acc_full"]:
        df[c] = pd.to_numeric(df[c], errors="coerce")

    return df[cols]

def _read_result_json(path: Path) -> Dict:
    if path.exists():
        try:
            return json.loads(path.read_text(encoding="utf-8"))
        except Exception:
            return {}
    return {}

def _gather_algo_runs(algo_dir: Path) -> Dict[str, Union[List[Tuple[int, Path, Path]], Optional[Path]]]:
    """
    识别一个算法目录下的多 seed 结构：
      - seeds: [(seed, gen_csv, result_json), ...]
      - avg: algo_dir/gen_log_avg.csv（若存在）
      - single: algo_dir/gen_log.csv（单 seed 时）
    """
    out = {"seeds": [], "avg": None, "single": None}

    avg_csv = algo_dir / "gen_log_avg.csv"
    if avg_csv.exists():
        out["avg"] = avg_csv

    # 多 seed 子目录
    for p in sorted(algo_dir.glob("seed_*")):
        try:
            seed = int(p.name.split("_", 1)[1])
        except Exception:
            continue
        gen_csv = p / "gen_log.csv"
        res_json = p / "result.json"
        if gen_csv.exists():
            out["seeds"].append((seed, gen_csv, res_json))

    # 单 seed
    if not out["seeds"]:
        gen_csv = algo_dir / "gen_log.csv"
        if gen_csv.exists():
            out["single"] = gen_csv

    return out


In [4]:
%cd ../../

/scratch/wx2178/SNN/DarwinNeuron


/scratch/wx2178/.conda/envs/cns/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
# ===== Cell 3: Load all gen logs (with seeds) into one long table =====

def load_all_runs_long(db: str, pids: Iterable[int], out_root: Path) -> pd.DataFrame:
    rows = _rows_for_problems(db, pids)
    records = []

    for run_id, ptype, pid, algoname, result_file in rows:
        algo = algoname.upper()
        run_dir = Path(result_file).parent           # 可能是 .../ALGO 或 .../ALGO/seed_S
        algo_dir = run_dir if run_dir.name.upper() == algo else run_dir.parent
        layout = _gather_algo_runs(algo_dir)

        if layout["seeds"]:
            # 多 seed：逐 seed 读取
            for seed, gen_csv, _res_json in layout["seeds"]:
                df = _read_gen_log(gen_csv)
                if df.empty: 
                    continue
                df.insert(0, "seed", seed)
                df.insert(0, "algorithm", algo)
                df.insert(0, "problem_id", pid)
                df.insert(0, "run_id", run_id)
                records.append(df)

            # 平均曲线（若需要画图或参考）
            if layout["avg"] is not None and layout["avg"].exists():
                dfa = _read_gen_log(layout["avg"])
                if not dfa.empty:
                    dfa.insert(0, "seed", "avg")
                    dfa.insert(0, "algorithm", algo)
                    dfa.insert(0, "problem_id", pid)
                    dfa.insert(0, "run_id", run_id)
                    records.append(dfa)
        else:
            # 单 seed
            gen_csv = layout["single"]
            if gen_csv is not None and gen_csv.exists():
                df = _read_gen_log(gen_csv)
                if not df.empty:
                    df.insert(0, "seed", 0)
                    df.insert(0, "algorithm", algo)
                    df.insert(0, "problem_id", pid)
                    df.insert(0, "run_id", run_id)
                    records.append(df)

    if not records:
        cols = ["run_id","problem_id","algorithm","seed","gen","n_evals","epoch","batch",
                "best_f","best_acc","avg_acc","wall_time","epoch_acc_full","x_file","x_epoch_file"]
        return pd.DataFrame(columns=cols)

    out = pd.concat(records, ignore_index=True)
    # 存一下长表（分 problem 存也行）
    (out_root / "analysis").mkdir(parents=True, exist_ok=True)
    out.to_csv(out_root / "analysis" / "all_runs_long.csv", index=False)
    out
    return out

df_long = load_all_runs_long(DB, PROBLEM_IDS, OUT_ROOT)
df_long.head()


In [9]:
import pandas as pd

all_rows = "data/MethodComparisonRuns/analysis/all_runs_long.csv"
output_file = "data/MethodComparisonRuns/analysis/runs_summary.csv"

df = pd.read_csv(all_rows)

# 1) 每个 (problem_id, algorithm, seed, epoch) 取 gen 最大的一行
last_per_epoch = (
    df.sort_values(["problem_id", "algorithm", "seed", "epoch", "gen"])
      .groupby(["problem_id", "algorithm", "seed", "epoch"], as_index=False)
      .tail(1)
)

# 2) 给 epoch_acc_full 补最近的非空值
# 方法：对每个组合 (problem_id, algorithm, seed)，按 epoch 升序，
#      用 ffill(bfill) 填充 epoch_acc_full
last_per_epoch["epoch_acc_full_filled"] = (
    last_per_epoch.sort_values(["problem_id", "algorithm", "seed", "epoch"])
    .groupby(["problem_id", "algorithm", "seed"])["epoch_acc_full"]
    .ffill()
    .bfill()
)

# 3) 保存结果
last_per_epoch.to_csv(output_file, index=False)

print(f"Saved summary to {output_file}, shape={last_per_epoch.shape}")
print(last_per_epoch.head())


Saved summary to data/MethodComparisonRuns/analysis/runs_summary.csv, shape=(28208, 16)
      run_id  problem_id algorithm seed  gen  n_evals  epoch  batch    best_f  \
7636     252           0     CMAES    0    2       70      0      2  0.891866   
7639     252           0     CMAES    0    5      220      1      2  0.686298   
7642     252           0     CMAES    0    8      370      2      2  0.553679   
7645     252           0     CMAES    0   11      520      3      2  0.484145   
7648     252           0     CMAES    0   14      670      4      2  0.438862   

      best_acc   avg_acc   wall_time  epoch_acc_full  \
7636  0.529070  0.532248   33.970935        0.505000   
7639  0.534884  0.518992  107.090044        0.535000   
7642  0.635659  0.583992  186.231977        0.597500   
7645  0.693798  0.605620  264.746003        0.692500   
7648  0.734496  0.639961  341.486718        0.716667   

                                                 x_file  \
7636  data/MethodComparisonRu

In [13]:
import pandas as pd

all_rows = "data/MethodComparisonRuns/analysis/runs_summary.csv"
output_file = "data/MethodComparisonRuns/analysis/runs_summary_short.csv"

# 读入 CSV
df = pd.read_csv(all_rows)

# 只保留 epoch 等于 99 或 100 的行
df_filtered = pd.concat([
    df[df["epoch"] == 100],
    df[(df["algorithm"] == "SGD") & (df["epoch"] == 99)]
])
df_filtered = df_filtered[df_filtered["seed"] != "avg"].copy()

# 存成新的 CSV
df_filtered.to_csv(output_file, index=False)

print(f"Filtered rows written to {output_file}, shape={df_filtered.shape}")
print(df_filtered.head())


Filtered rows written to data/MethodComparisonRuns/analysis/runs_summary_short.csv, shape=(215, 16)
     run_id  problem_id algorithm seed  gen  n_evals  epoch  batch    best_f  \
100     252           0     CMAES    0  300    14970    100      0  0.123908   
201     252           0     CMAES    1  300    14970    100      0  0.107587   
378     252           0     CMAES    5  300    14970    100      0  0.106792   
479     252           0     CMAES    6  300    14970    100      0  0.151216   
615     249           0        DE    0  300    15000    100      0  0.402712   

     best_acc   avg_acc    wall_time  epoch_acc_full  \
100  0.928571  0.795000  7674.187000             NaN   
201  0.916667  0.845238  8079.311826             NaN   
378  0.922619  0.887976  1981.259895             NaN   
479  0.916667  0.764881  1967.786267             NaN   
615  0.660714  0.655952  5189.215234             NaN   

                                                x_file x_epoch_file  \
100  data/M

In [6]:
# ===== Cell 5: Load landscape features from SQLite =====

def list_tables(db: str) -> List[str]:
    with sqlite3.connect(db) as con:
        q = "SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;"
        return [r[0] for r in con.execute(q).fetchall()]

def infer_feature_table_and_key(db: str) -> Tuple[Optional[str], Optional[str]]:
    """
    尝试寻找一个包含 problem id 的“特征表”：
      - 优先包含列：problem_id 或 nn_problem_id
      - 排除运行表：runs/algorithm/termination 等
    """
    cand_keys = ["problem_id","nn_problem_id","pid","id"]
    bad_names = {"runs","run","algorithm","termination","gen_log","result","pymoo_history"}
    tbs = list_tables(db)
    for tb in tbs:
        if tb.lower() in bad_names:
            continue
        try:
            with sqlite3.connect(db) as con:
                info = con.execute(f"PRAGMA table_info({tb});").fetchall()
                cols = [r[1] for r in info]
        except Exception:
            continue
        key = None
        for k in cand_keys:
            if k in cols:
                key = k
                break
        if key is not None:
            # 进一步简单启发式：包含一些“明显是特征”的列名
            feature_like = [c for c in cols if c not in {key} and not c.endswith("_id")]
            if len(feature_like) >= 2:
                return tb, key
    return None, None

def load_features(db: str,
                  table_name: Optional[str] = None,
                  key_col: Optional[str] = None) -> pd.DataFrame:
    if table_name is None or key_col is None:
        tb, key = infer_feature_table_and_key(db)
        table_name = table_name or tb
        key_col = key_col or key

    if not table_name or not key_col:
        raise RuntimeError("无法自动推断 feature 表名/键列。请手工设置 FEATURE_TABLE_NAME 和 FEATURE_KEY_COLUMN。")

    with sqlite3.connect(db) as con:
        df = pd.read_sql_query(f"SELECT * FROM {table_name}", con)

    # 规范 key 名为 problem_id
    if key_col != "problem_id":
        df = df.rename(columns={key_col: "problem_id"})
    return df

# 自动/显式加载
feat_df = load_features(DB, FEATURE_TABLE_NAME, FEATURE_KEY_COLUMN)
print("features shape:", feat_df.shape)
feat_df.head()


features shape: (216, 4)


,problem_id,function_idx,instance_idx,dim
0,1,1,1,2
1,2,1,2,2
2,3,1,3,2
3,4,1,1,160
4,5,1,2,160


In [7]:
# ===== Cell 6: Merge features & performance, do correlations and baselines =====

def best_algo_per_problem(avg_df: pd.DataFrame) -> pd.DataFrame:
    """为每个 problem 选出 mean_best_f 最小的算法，得到 (problem, best_algo, best_f, best_acc, ...)"""
    rows = []
    for pid, g in avg_df.groupby("problem"):
        g2 = g.sort_values("mean_best_f", ascending=True).iloc[0]
        rows.append({
            "problem_id": pid,
            "best_algo":  g2["algorithm"],
            "best_f":     g2["mean_best_f"],
            "best_acc":   g2["mean_best_acc"],
            "best_wall":  g2["mean_wall_time"],
            "finished_ratio": g2["finished_ratio"],
            "epochs_done_mean": g2["epochs_done_mean"],
        })
    return pd.DataFrame(rows)

def melt_algo_performance(avg_df: pd.DataFrame) -> pd.DataFrame:
    """
    另一种：把 (problem, algorithm) 的均值性能保留，用来做 per-algorithm 回归。
    返回列：problem_id, algorithm, mean_best_f, mean_best_acc, ...
    """
    m = avg_df.rename(columns={"problem":"problem_id"}).copy()
    return m

# 1) 取问题级标签（最佳算法） & 合并 features
label_df = best_algo_per_problem(avg_df)
meta_cls_df = feat_df.merge(label_df, on="problem_id", how="inner")
print("meta classification table:", meta_cls_df.shape)
meta_cls_df.head()

# 2) 简单相关性（features ↔ best_f / best_acc）
num_cols = meta_cls_df.select_dtypes(include=[np.number]).columns.tolist()
corr_target_cols = ["best_f","best_acc"]
corrs = {}
for tgt in corr_target_cols:
    if tgt in meta_cls_df.columns:
        corrs[tgt] = meta_cls_df[num_cols].corr()[tgt].sort_values(ascending=False)
        print(f"\n=== Correlations with {tgt} ===")
        display(corrs[tgt].to_frame(f"corr_with_{tgt}"))

# 3) Leave-One-Problem-Out 分类 baseline：预测 best_algo
if len(meta_cls_df) >= 3:
    # 准备特征/标签
    feature_cols = [c for c in meta_cls_df.columns 
                    if c not in {"problem_id","best_algo","best_f","best_acc","best_wall","finished_ratio","epochs_done_mean"} 
                    and pd.api.types.is_numeric_dtype(meta_cls_df[c])]
    X = meta_cls_df[feature_cols].values
    y = meta_cls_df["best_algo"].values

    # LOO-CV
    uniq_probs = meta_cls_df["problem_id"].unique()
    preds, trues = [], []
    for pid in uniq_probs:
        train_idx = meta_cls_df["problem_id"] != pid
        test_idx  = meta_cls_df["problem_id"] == pid

        pipe = Pipeline([
            ("scaler", StandardScaler(with_mean=True, with_std=True)),
            ("clf", RandomForestClassifier(n_estimators=300, random_state=42))
        ])
        pipe.fit(X[train_idx], y[train_idx])
        yhat = pipe.predict(X[test_idx])
        preds.extend(yhat.tolist())
        trues.extend(y[test_idx].tolist())

    acc = accuracy_score(trues, preds)
    print(f"\n[LOO] Best-optimizer classification accuracy = {acc:.3f}  "
          f"(n_problems={len(uniq_probs)}, classes={len(np.unique(y))})")

# 4) 每算法回归 baseline（预测 mean_best_f）
perf_long = melt_algo_performance(avg_df)     # problem_id, algorithm, mean_best_f, ...
meta_reg_df = perf_long.merge(feat_df, on="problem_id", how="inner")

if len(meta_reg_df["problem_id"].unique()) >= 3:
    feature_cols = [c for c in meta_reg_df.columns 
                    if c not in {"problem_id","algorithm","mean_best_f","mean_best_acc","mean_epoch_acc_full",
                                 "mean_wall_time","seeds","finished_ratio","epochs_done_mean"}
                    and pd.api.types.is_numeric_dtype(meta_reg_df[c])]

    uniq_probs = meta_reg_df["problem_id"].unique()
    maes, r2s = [], []
    for pid in uniq_probs:
        trn = meta_reg_df["problem_id"] != pid
        tst = meta_reg_df["problem_id"] == pid

        Xtr = meta_reg_df.loc[trn, feature_cols].values
        ytr = meta_reg_df.loc[trn, "mean_best_f"].values
        Xte = meta_reg_df.loc[tst, feature_cols].values
        yte = meta_reg_df.loc[tst, "mean_best_f"].values

        pipe = Pipeline([
            ("scaler", StandardScaler(with_mean=True, with_std=True)),
            ("reg", RandomForestRegressor(n_estimators=400, random_state=42))
        ])
        pipe.fit(Xtr, ytr)
        yhat = pipe.predict(Xte)
        maes.append(mean_absolute_error(yte, yhat))
        r2s.append(r2_score(yte, yhat))

    print(f"[LOO] mean_best_f regression  |  MAE={np.mean(maes):.4f}  R^2={np.mean(r2s):.3f}")


NameError: name 'avg_df' is not defined